In [10]:
pip install selenium


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install pydrive


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import os
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException


In [35]:

# Set up directory for saving images
folder_name = "tropical vacation"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Scroll settings
sleepTime = 7  # Increase to allow images to load
max_scrolls = 50  # Prevent infinite loops
max_images = 150  # Target number of images

# Pinterest URL
var = 'vacation poses'
url = f'https://www.pinterest.com/search/pins/?q={var}'

# Set up Chrome options
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])

# Start WebDriver
service = Service("/opt/homebrew/bin/chromedriver")  # Update this path if needed
driver = webdriver.Chrome(service=service, options=options)

# Open the webpage
driver.get(url)
time.sleep(5)  # Allow time for page to load

# Scrolling logic with user interaction
image_urls = set()
scroll_attempts = 0

while len(image_urls) < max_images and scroll_attempts < max_scrolls:
    # Scroll all the way down
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(sleepTime)

    # Simulate user interaction (hovering over an image)
    images = driver.find_elements(By.TAG_NAME, "img")
    if images:
        hover = ActionChains(driver).move_to_element(images[-1])  # Hover over the last image
        hover.perform()
        time.sleep(2)  # Give time for new content to load

    # Refresh image elements
    try:
        images = driver.find_elements(By.TAG_NAME, "img")
        for img in images:
            src = img.get_attribute('src')
            if src and 'pinimg.com' in src:
                image_urls.add(src)
    except StaleElementReferenceException:
        print("Encountered stale element, skipping this batch.")

    print(f'Scrolling... Collected {len(image_urls)} images so far')
    scroll_attempts += 1

# Convert low-res URLs to high-res
def get_high_res_url(src):
    resolutions = ['/originals/', '/564x/', '/736x/']
    for res in resolutions:
        high_res_url = src.replace('/60x60/', res).replace('/236x/', res)
        if requests.head(high_res_url).status_code == 200:
            return high_res_url
    return src  # Return original if higher resolution is not available

image_urls = list(map(get_high_res_url, image_urls))

# Download and save images
for img_url in image_urls:
    try:
        filename = os.path.basename(img_url).replace("/", "_")
        file_path = os.path.join(folder_name, filename)

        response = requests.get(img_url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print(f"Saved: {file_path}")
        else:
            print(f"Failed to download: {img_url}")

    except Exception as e:
        print(f"Error saving {img_url}: {e}")

# Close the browser
driver.quit()


Scrolling... Collected 32 images so far
Scrolling... Collected 45 images so far
Scrolling... Collected 59 images so far
Scrolling... Collected 71 images so far
Scrolling... Collected 84 images so far
Scrolling... Collected 97 images so far
Scrolling... Collected 111 images so far
Scrolling... Collected 123 images so far
Scrolling... Collected 136 images so far
Scrolling... Collected 148 images so far
Scrolling... Collected 161 images so far
Saved: tropical vacation/1c5ce48714547c51b452da747a3d9e44.jpg
Saved: tropical vacation/d492e037267dfda7ef8ab4a53242f490.jpg
Saved: tropical vacation/0e22b1bbfbb77c99c809518cf554070c.jpg
Saved: tropical vacation/7510a4afdee4ccb95f308d25278bfe47.jpg
Saved: tropical vacation/a05d24a5c4bbf611ba546bcc1fbf4645.jpg
Saved: tropical vacation/3aa6ed25bda7e8a60a49b67571c84f81.jpg
Saved: tropical vacation/db66df878d92d9eb3297021533834fa1.jpg
Saved: tropical vacation/df863a138d37dbc266510cc192e6b02e.jpg
Saved: tropical vacation/21225b103a10a9439c199f79aec03a2c.j